# ABL1 complex modeling with water

This notebook shows the modeling of tyrosine kinase inhibitors into ABL1 using functionalities from KinoML from the commit [b252ff6](https://github.com/openkinome/kinoml/commit/b252ff67a35b9149e704d51b7c838dcf1e9fb10f).

In [1]:
from kinoml.core.ligands import SmilesLigand
from kinoml.core.components import BaseProtein
from kinoml.core.systems import ProteinLigandComplex
from kinoml.features.complexes import OEKLIFSKinaseHybridDockingFeaturizer

In [2]:
# smiles from chembl
tkis = {'axitinib': 'CNC(=O)c1ccccc1Sc1ccc2c(/C=C/c3ccccn3)n[nH]c2c1',
        'bosutinib': 'COc1cc(Nc2c(C#N)cnc3cc(OCCCN4CCN(C)CC4)c(OC)cc23)c(Cl)cc1Cl',
        'dasatinib': 'Cc1nc(Nc2ncc(C(=O)Nc3c(C)cccc3Cl)s2)cc(N2CCN(CCO)CC2)n1',
        'imatinib': 'Cc1ccc(NC(=O)c2ccc(CN3CCN(C)CC3)cc2)cc1Nc1nccc(-c2cccnc2)n1',
        'nilotinib': 'Cc1cn(-c2cc(NC(=O)c3ccc(C)c(Nc4nccc(-c5cccnc5)n4)c3)cc(C(F)(F)F)c2)cn1',
        'ponatinib': 'Cc1ccc(C(=O)Nc2ccc(CN3CCN(C)CC3)c(C(F)(F)F)c2)cc1C#Cc1cnc2cccnn12',
        'erlotinib': 'C#Cc1cccc(Nc2ncnc3cc(OCCOC)c(OCCOC)cc23)c1',
        'gefitinib': 'COc1cc2ncnc(Nc3ccc(F)c(Cl)c3)c2cc1OCCCN1CCOCC1'}

In [3]:
docking_featurizer = OEKLIFSKinaseHybridDockingFeaturizer(loop_db="~/.OpenEye/rcsb_spruce.loop_db")

In [4]:
for tki, smiles in tkis.items():
    smiles_ligand = SmilesLigand(smiles=smiles, name=tki)
    base_protein = BaseProtein(name='ABL1')
    base_protein.klifs_kinase_id = 392
    kinase_ligand_complex = ProteinLigandComplex(components=[base_protein, smiles_ligand])
    system = docking_featurizer.featurize(kinase_ligand_complex)

All ABL1 structures in the main `data` directory were automatically standardized to the kinase domain according to UniProt, i.e., all missing loops built, mutated back to the wild type sequence, truncated according to UniProt kinase domain, capped termini and renumbered according to UniProt. Co-crystallized water were checked for clashes with the ligand. An aromatic nitrogen of ponatinib is wrongly protonated leading to a flipped moiety. The substituted aromatic ring of gefitinib is flipped compared to the pose in the ligand template [2itz](https://www.rcsb.org/structure/2itz).

|tki      |kinase template|ligand template|ligand co-crystallized|comment                                     |
|---------|---------------|---------------|----------------------|--------------------------------------------|
|axitinib |4twp           |4twp           |+                     |protonated ASP in binding site              |
|bosutinib|3ue4           |3ue4           |+                     |piperazine 2+                               |
|dasatinib|4xey           |4xey           |+                     |no water                                    |
|imatinib |2hyy           |2hyy           |+                     |piperazine 2+                               |
|nilotinib|3cs9           |3cs9           |+                     |                                            |
|ponatinib|2hiw           |3oxz           |+                     |protonated aromatic nitrogen, flipped moiety|
|erlotinib|2g1t           |4hjo           |+                     |                                            |
|gefitinib|2f4j           |2itz           |+                     |ring flipped compared to ligand template    |